# Amazon Bedrock AgentCore Gateway Integration with Amazon Bedrock AgentCore Runtime

## Overview
[Amazon Bedrock AgentCore Gateway](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway.html) provides customers a way to turn their existing AWS Lambda functions & APIs (OpenAPI and Smithy) into fully-managed MCP servers without needing to manage infra or hosting. [Amazon Bedrock AgentCore Runtime](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html) provides a secure, serverless and purpose-built hosting environment for deploying and running AI agents or tools. In this tutorial we will integrate Amazon Bedrock AgentCore Gateway with AgentCore runtime and [Strands agents](https://strandsagents.com/latest/). 

### Tutorial Details


| Information          | Details                                                   |
|:---------------------|:----------------------------------------------------------|
| Tutorial type        | Interactive                                               |
| AgentCore components | AgentCore Gateway, AgentCore Identity, AgentCore Runtime  |
| Agentic Framework    | Strands Agents                                            |
| Gateway Target type  | AWS Lambda, OpenAPI target                                |
| Inbound Auth IdP     | AWS IAM                                                   |
| Outbound Auth        | AWS IAM (AWS Lambda), API Key (OpenAPI target)            |
| LLM model            | Anthropic Claude Sonnet 3.7, Amazon Nova Pro              |
| Tutorial components  | Creating AgentCore Gateway and Invoking AgentCore Gateway |
| Tutorial vertical    | Cross-vertical                                            |
| Example complexity   | Medium                                                    |
| SDK used             | boto3                                                     |


### Tutorial Architecture
In this tutorial we will transform operations defined in AWS lambda function & Restful API into MCP tools and host it in Bedrock AgentCore Gateway. We will demonstrate the ingress auth using AWS IAM credentials in AWS Sigv4 format. We will deploy a Strands Agent utilizing AgentCore Gateway tools on AgentCore runtime. 

For demonstration purposes, we will use a Strands Agent using [Amazon Bedrock](https://aws.amazon.com/bedrock/) models. 

<center>

![runtime gateway](./images/runtime_gateway.png)

</center>

## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
import os
import sys

if "__file__" in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()

# Navigate up one level to access the shared utils module
# The utils.py file contains helper functions used across multiple tutorials
utils_dir = os.path.abspath(os.path.join(current_dir, ".."))

# Add the utils directory to Python's module search path
# This allows us to import utils from the parent directory
sys.path.insert(0, utils_dir)

# Import the utils module which contains helper functions for gateway management
import utils

In [ ]:
# Import all necessary libraries for the tutorial
from bedrock_agentcore_starter_toolkit import Runtime 
from bedrock_agentcore_starter_toolkit.operations.runtime import destroy_bedrock_agentcore
from deploy_cloudformation import deploy_stack, delete_stack
from pathlib import Path
from strands import Agent  
from strands.models import BedrockModel  
from strands.tools.mcp.mcp_client import MCPClient 
from streamable_http_sigv4 import streamablehttp_client_with_sigv4  
import boto3  
import getpass 
import json
import logging  
import uuid 

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [ ]:
session = boto3.Session()

credentials = session.get_credentials()

region = session.region_name

cf_client = boto3.client(
    "cloudformation", region_name=region
)

agentcore_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=region,
)

identity_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=region,
)

s3_client = session.client('s3')

sts_client = session.client('sts')
account_id = sts_client.get_caller_identity()["Account"]

In [ ]:
# Define configuration variables for the tutorial resources

# CloudFormation stack configuration
stack_name = "customer-support-lambda-stack"  # Name of the stack that creates Lambda and DynamoDB
template_file = "cloudformation/customer_support_lambda.yaml"  # Path to CloudFormation template

# Gateway and target names
gateway_name = "customer-support-gatewat"  # Name for the AgentCore Gateway
open_api_target_name = 'DemoOpenAPITargetS3NasaMars'  # Name for the NASA API gateway target
lambda_target_name = "LambdaUsingSDK"  # Name for the Lambda function gateway target

# S3 bucket configuration for storing OpenAPI specifications
unique_s3_name = str(uuid.uuid4())  # Generate a globally unique identifier for the bucket
bucket_name = f'agentcore-gateway-{unique_s3_name}'  # Prefix with 'agentcore-gateway' for clarity
file_path = 'openapi-specs/nasa_mars_insights_openapi.json'  # Local path to OpenAPI spec
object_key = 'nasa_mars_insights_openapi.json'  # S3 object key (filename in the bucket)

# Credential provider name for NASA API authentication
api_key_credential_provider_name = "NasaInsightAPIKey"  # Will store the NASA API key securely

# Agent configuration
agent_name = "customer_support_gateway"  # Name for the deployed agent in AgentCore Runtime

# Store the unique S3 name for use in other notebooks if needed
%store unique_s3_name

## Step 1: Deploy AWS Lambda & Amazon DynamoDB

### AWS CloudFormation Stack Resources

The [AWS CloudFormation](https://docs.aws.amazon.com/AWSCloudFormation/latest/UserGuide/Welcome.html) template will deploy the following AWS resources:

1. **AgentCoreRuntimeExecutionRole**: Execution role for AgentCore Runtime with permissions for:
   - ECR image access for container deployment
   - CloudWatch Logs for monitoring and debugging
   - X-Ray tracing for observability
   - Bedrock model invocation for AI capabilities
   - Gateway invocation to call MCP tools
   - Workload identity token generation

2. **GatewayAgentCoreRole**: Execution role for AgentCore Gateway with permissions for:
   - Lambda function invocation
   - S3 access for OpenAPI specifications
   - Secrets Manager access for credential retrieval
   - All AgentCore and Bedrock operations
   - Includes enhanced trust policy with confused deputy protection

3. **CustomerSupportLambda**: Main Lambda function that provides:
   - `get_customer_profile`: Retrieves customer information by ID, email, or phone
   - `check_warranty_status`: Validates product warranty using serial numbers
   
4. **PopulateDataFunction**: Custom resource Lambda that seeds initial data into DynamoDB tables

5. **CustomerProfileTable**: Stores customer information with:
   - Primary key: `customer_id`
   - Global Secondary Indexes on `email` and `phone` for flexible lookups
   - Contains 5 sample customer profiles

6. **WarrantyTable**: Stores product warranty information with:
   - Primary key: `serial_number`
   - Global Secondary Index on `customer_id` for customer-based queries
   - Contains 8 sample warranty records

The stack exports three ARNs for use in the tutorial:
- `CustomerSupportLambdaArn`: ARN of the Lambda function (gateway target)
- `GatewayAgentCoreRoleArn`: ARN for gateway execution
- `AgentCoreRuntimeExecutionRoleArn`: ARN for runtime agent deployment

All resources include proper tagging for cost tracking and management.

In [ ]:
# Deploy the CloudFormation stack
lambda_arn, gateway_role_arn, runtime_execution_role_arn = deploy_stack(
    stack_name=stack_name,
    template_file=template_file,
    region=region,
    cf_client=cf_client
)

## Step 2: Creat AgentCore Gateway using IAM as Authorizer for inbound requests

AgentCore Gateway has made a significant advancement in its authentication capabilities by introducing [AWS IAM](https://aws.amazon.com/iam/) support for inbound authentication, expanding beyond its previous OAuth-only approach. While the [MCP protocol](https://modelcontextprotocol.io/docs/getting-started/intro) specification traditionally requires OAuth tokens for authentication, this new feature allows customers to use AWS IAM credentials for inbound requests to their MCP servers, addressing a crucial enterprise requirement.

Previously, gateway developers could only configure CUSTOM_JWT as the authorizer type, requiring OAuth token-based authentication for all inbound requests. With this release, developers can now choose AWS_IAM as the authorizer type, enabling AWS Signature Version 4 (SigV4) authentication for inbound MCP requests. 

The implementation introduces two key IAM components:

- New action `bedrock-agentcore:InvokeGateway` for gateway request authorization
- New condition key `bedrock-agentcore:GatewayAuthorizerType` for controlling gateway creation based on authorizer type

In [ ]:
# Create an AgentCore Gateway with AWS IAM as the authorizer
# This is the key feature - using AWS_IAM instead of CUSTOM_JWT for authentication
create_response = agentcore_client.create_gateway(
    name=gateway_name,
    roleArn=gateway_role_arn,  
    protocolType="MCP", 
    authorizerType="AWS_IAM", 
    description="AgentCore Gateway with AWS Lambda target type using AWS IAM for ingress auth",
)
logger.info(f"Gateway created: {create_response}")

gateway_id = create_response["gatewayId"]
gateway_url = create_response["gatewayUrl"]
logger.info(f"Gateway ID: {gateway_id}")
logger.info(f"Gateway URL: {gateway_url}")

## Step 3: Transform Customer Support AWS Lambda into MCP tools using Bedrock AgentCore Gateway

<center>

![prereq](./images/lambda_dynamodb.png)

</center>

In [ ]:
# Configure the Lambda function as a gateway target
# This transforms Lambda function operations into MCP tools that AI agents can use
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": lambda_arn, 
            
            "toolSchema": {
                "inlinePayload": [
                    {
                        # First tool: retrieve customer profile information
                        "name": "get_customer_profile",
                        "description": "Retrieve customer profile using customer ID, email, or phone number",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                # Customer can be looked up by ID, email, or phone
                                "customer_id": {"type": "string"},
                                "email": {"type": "string"},
                                "phone": {"type": "string"},
                            },
                            # At minimum, customer_id is required
                            "required": ["customer_id"],
                        },
                    },
                    {
                        # Second tool: check warranty status for products
                        "name": "check_warranty_status",
                        "description": "Check the warranty status of a product using its serial number and optionally verify via email",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                # Serial number uniquely identifies the product
                                "serial_number": {"type": "string"},
                                # Customer email can be used for verification
                                "customer_email": {"type": "string"},
                            },
                            # Serial number is mandatory
                            "required": ["serial_number"],
                        },
                    },
                ]
            },
        }
    }
}

# Configure how the gateway authenticates to the Lambda function
# Using the gateway's IAM role (gateway_role_arn) to invoke Lambda
credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]

# Create the gateway target - this makes the Lambda function available as MCP tools
response = agentcore_client.create_gateway_target(
    gatewayIdentifier=gateway_id,  
    name=lambda_target_name,  
    description="Lambda Target using SDK",
    targetConfiguration=lambda_target_config,  
    credentialProviderConfigurations=credential_config,  
)

## Step 4: Transform NASA Open APIs into MCP tools using Bedrock AgentCore Gateway


<center>

![prereq](./images/nasa.png)

</center>

We are going to have a Mars Weather agent getting weather data from Nasa's Open APIs. You will need to register for Nasa Insight API [here](https://api.nasa.gov/). It's free! Once you register, you will get an API Key in your email. Use the API key to configure the credentials provider for creating the OpenAPI target.



### Step 4.1 Create API Credential Provider - Amazon Bedrock AgentCore Identity

In [ ]:
# Securely prompt the user for their NASA API Key
# getpass hides the input to prevent it from being displayed on screen
nasa_api_key = getpass.getpass(prompt='Enter your NASA API Key: ')

In [ ]:
if not nasa_api_key:
    logger.error("NASA API Key is required. Please run the cell above and enter your API key.")
    raise ValueError("NASA API Key is required")

# Create a credential provider in AgentCore Identity to securely store the API key
# This allows the gateway to authenticate to the NASA API without exposing the key
response = identity_client.create_api_key_credential_provider(
    name=api_key_credential_provider_name,  
    apiKey=nasa_api_key,
)

logger.info(f"Credential provider response: {response}")

credential_provider_arn = response['credentialProviderArn']
logger.info(f"Egress Credentials provider ARN: {credential_provider_arn}")

### Step 4.2: Create Amazon S3 Bucket to upload [NASA OpenAPI Spec](./openapi-specs/nasa_mars_insights_openapi.json)

In [ ]:
try:
    # Create an S3 bucket to store the OpenAPI specification file
    if region == "us-east-1":
        # us-east-1 doesn't require LocationConstraint
        s3bucket = s3_client.create_bucket(
            Bucket=bucket_name
        )
    else:
        # All other regions need LocationConstraint to specify the region
        s3bucket = s3_client.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={
                'LocationConstraint': region
            }
        )
    
    # Upload the OpenAPI specification JSON file to S3
    # The gateway will read this file to understand the NASA API structure
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(
            Bucket=bucket_name,  
            Key=object_key,  
            Body=file_data  
        )

    openapi_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'Uploaded object S3 URI: {openapi_s3_uri}')
except Exception as e:
    print(f'Error uploading file: {e}')

### Step 4.3 Configure outbound auth and Create the gateway target


In [ ]:
# Configure the NASA OpenAPI target
# This will transform NASA's Mars Insight API into MCP tools
nasa_openapi_s3_target_config = {
    "mcp": {
        "openApiSchema": {
            "s3": {
                "uri": openapi_s3_uri
            }
        }
    }
}

# Configure API key authentication for outbound requests to NASA
# The gateway will attach the API key to every request to the NASA API
api_key_credential_config = [
    {
        "credentialProviderType": "API_KEY",
        "credentialProvider": {
            "apiKeyCredentialProvider": {
                # NASA expects the API key as a query parameter named "api_key"
                "credentialParameterName": "api_key", 
                
                # ARN of the credential provider we created earlier
                "providerArn": credential_provider_arn,
                
                # NASA API expects the key in the query string (not in headers)
                "credentialLocation": "QUERY_PARAMETER",  # Options: "HEADER" or "QUERY_PARAMETER"
                
                # Note: credentialPrefix (like "Basic" or "Bearer") is used for header-based auth
                # "credentialPrefix": " "  # Uncomment if using header-based token auth
            }
        }
    }
]

# Create the OpenAPI gateway target
# This makes all NASA API operations available as MCP tools
response = agentcore_client.create_gateway_target(
    gatewayIdentifier=gateway_id,  
    name=open_api_target_name,  
    description='OpenAPI Target with S3Uri using SDK',
    targetConfiguration=nasa_openapi_s3_target_config,  
    credentialProviderConfigurations=api_key_credential_config 
)

## Step 5: Call the MCP tools from Strands Agent

#### Support for AWS IAM Authentication in MCP Client SDKs
Support for AWS IAM Authentication in MCP Client SDKs

While AWS IAM authentication is now supported for inbound requests to AgentCore Gateway, it's important to note that current open-source MCP Client SDKs have limited support for SigV4 authentication, particularly for streamable HTTP connections. However, AWS has provided a solution through the "Run Model Context Protocol (MCP) servers with AWS Lambda" project, which includes a crucial extension for SigV4 authentication with streaming HTTP connections.

This implementation, available at the [AWS Labs GitHub repository](https://github.com/awslabs/run-model-context-protocol-servers-with-aws-lambda/tree/main), bridges the authentication gap for streaming connections and can be seamlessly integrated with popular agentic frameworks like Strands or LangChain. The StreamableHTTPTransportWithSigV4 class extends the standard MCP transport layer to handle AWS SigV4 signing while maintaining streaming capabilities, making it compatible with AgentCore Gateway's new IAM authentication feature.

In [ ]:
def create_streamable_http_transport_sigv4(mcp_url: str, service_name: str, region: str):
    """
    Create a streamable HTTP transport with AWS SigV4 authentication.
    
    This function creates an MCP client transport that uses AWS Signature Version 4 (SigV4)
    to authenticate requests. This is necessary because standard MCP clients don't natively
    support AWS IAM authentication, and this bridges that gap.
    
    Args:
        mcp_url (str): The URL of the MCP gateway endpoint
        service_name (str): The AWS service name for SigV4 signing (typically "bedrock-agentcore")
        region (str): The AWS region where the gateway is deployed
    
    Returns:
        StreamableHTTPTransportWithSigV4: A transport instance configured for SigV4 auth
    
    Example:
        >>> transport = create_streamable_http_transport_sigv4(
        ...     mcp_url="https://gateway-id.gateway.region.aws.dev/mcp",
        ...     service_name="bedrock-agentcore",
        ...     region="us-west-2"
        ... )
    """
    # Get AWS credentials from the current boto3 session
    # These credentials will be used to sign requests with SigV4
    session = boto3.Session()
    credentials = session.get_credentials()
    
    # Create and return the custom transport with SigV4 signing capability
    return streamablehttp_client_with_sigv4(
        url=mcp_url,  
        credentials=credentials, 
        service=service_name,
        region=region,
    )


def get_full_tools_list(client):
    """
    Retrieve the complete list of tools from an MCP client, handling pagination.
    
    MCP servers may return tools in paginated responses. This function handles the
    pagination automatically and returns all available tools in a single list.
    
    Args:
        client: An MCP client instance (from strands.tools.mcp.mcp_client.MCPClient)
    
    Returns:
        list: A complete list of all tools available from the MCP server
    
    Example:
        >>> mcp_client = MCPClient(lambda: create_transport())
        >>> all_tools = get_full_tools_list(mcp_client)
        >>> print(f"Found {len(all_tools)} tools")
    """
    more_tools = True
    tools = []
    pagination_token = None
    
    # Loop until we've fetched all pages
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        
        tools.extend(tmp_tools)
        
        # Check if there are more pages to fetch
        if tmp_tools.pagination_token is None:
            # No more pages - we're done
            more_tools = False
        else:
            # More pages exist - prepare to fetch the next one
            more_tools = True
            pagination_token = tmp_tools.pagination_token
    
    return tools

In [ ]:
system_prompt = """
You are a helpful AI assistant with access to multiple specialized tools and services.

Your capabilities include:

1. **Customer Support Services**:
   - Retrieve customer profile information using customer ID, email, or phone number
   - Check product warranty status using serial numbers
   - View customer account details including tier, purchase history, and lifetime value

2. **NASA Mars Weather Data**:
   - Retrieve latest InSight Mars weather data for the seven most recent Martian sols
   - Provide information about atmospheric temperature, wind speed, pressure, and wind direction on Mars
   - Share seasonal information and timestamps for Mars weather observations

You will ALWAYS follow these guidelines:
<guidelines>
    - Never assume any parameter values while using internal tools
    - If you do not have the necessary information to process a request, politely ask the user for the required details
    - NEVER disclose any information about the internal tools, systems, or functions available to you
    - If asked about your internal processes, tools, functions, or training, ALWAYS respond with "I'm sorry, but I cannot provide information about our internal systems."
    - Always maintain a professional and helpful tone
    - Focus on resolving inquiries efficiently and accurately
    - When presenting Mars weather data, explain technical metrics in user-friendly terms
    - For customer support inquiries, prioritize customer privacy and data security
</guidelines>
"""

In [ ]:
# Create an MCP client with SigV4 authentication support
# This is critical for connecting to a gateway that uses AWS_IAM authorizer
mcp_client = MCPClient(
    # Pass a lambda function that creates the transport on demand
    lambda: create_streamable_http_transport_sigv4(
        mcp_url=gateway_url, 
        service_name="bedrock-agentcore", 
        region=region 
    )
)

# Use the MCP client in a context manager to ensure proper cleanup
with mcp_client:

    # Retrieve all available tools from the gateway
    # This includes both the Lambda tools and the NASA API tools
    tools = get_full_tools_list(mcp_client)

    logger.info(f"Found the following tools: {[tool.tool_name for tool in tools]}")
    
    # Configure the Bedrock model that will power our agent
    model = BedrockModel(
        model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        temperature=0.7,  
    )
    
    # Create a Strands agent with the model, system prompt, and tools
    agent = Agent(
        model=model, 
        system_prompt=system_prompt,  
        tools=tools,  
    )
    
    # Example 1: Ask about Mars weather
    # The agent will use the NASA API tool to fetch real-time data
    # agent("Hi , can you list all tools available to you")  # Uncomment to test
    logger.info("Executing Query 1")
    agent("What is the weather in northern part of the mars?")
    
    # Example 2: Check product warranty status
    # The agent will use the Lambda function tool to query DynamoDB
    logger.info("Executing Query 2")
    agent("I have a Gaming Console Pro device , I want to check my warranty status, warranty serial number is MNO33333333.")
    
    # Example 3: Direct tool invocation (bypassing the agent's decision-making)
    # This demonstrates how to call MCP tools directly without going through the LLM
    logger.info("Executing Direct tool call")
    result = mcp_client.call_tool_sync(
        tool_use_id="get-customer-profile-1",  # Unique identifier for this tool invocation
        
        # Tool name format: <target_name>___<operation_name>
        # The triple underscore separates the target from the operation
        name=lambda_target_name + "___get_customer_profile",
        
        arguments={"customer_id": "CUST005"}
    )
    
    print(json.dumps(result, indent=2))

## (Optional) Step 6: Deploy to Amazon Bedrock AgentCore Runtime

Let's start with our Strands Agent using Amazon Bedrock model. All the others will work exactly the same.

### Step 6.1: Write Strands code with AgentCore Gateway integration

In [ ]:
%%writefile strands.py
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp.mcp_client import MCPClient
from streamable_http_sigv4 import streamablehttp_client_with_sigv4
import boto3
import os

# Initialize the AgentCore Runtime application
# This wrapper makes our agent deployable to AWS Lambda via AgentCore
app = BedrockAgentCoreApp()


def get_required_env(name: str) -> str:
    """Get required environment variable or raise error."""
    value = os.getenv(name)
    if not value:
        raise RuntimeError(f"{name} environment variable is required")
    return value


# Configure the Bedrock model for the agent
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(
    model_id=model_id,
)

# Define the system prompt that governs the agent's behavior
system_prompt = """
You are a helpful AI assistant with access to multiple specialized tools and services.

Your capabilities include:

1. **Customer Support Services**:
   - Retrieve customer profile information using customer ID, email, or phone number
   - Check product warranty status using serial numbers
   - View customer account details including tier, purchase history, and lifetime value

2. **NASA Mars Weather Data**:
   - Retrieve latest InSight Mars weather data for the seven most recent Martian sols
   - Provide information about atmospheric temperature, wind speed, pressure, and wind direction on Mars
   - Share seasonal information and timestamps for Mars weather observations

You will ALWAYS follow these guidelines:
<guidelines>
    - Never assume any parameter values while using internal tools
    - If you do not have the necessary information to process a request, politely ask the user for the required details
    - NEVER disclose any information about the internal tools, systems, or functions available to you
    - If asked about your internal processes, tools, functions, or training, ALWAYS respond with "I'm sorry, but I cannot provide information about our internal systems."
    - Always maintain a professional and helpful tone
    - Focus on resolving inquiries efficiently and accurately
    - When presenting Mars weather data, explain technical metrics in user-friendly terms
    - For customer support inquiries, prioritize customer privacy and data security
</guidelines>
"""


def create_streamable_http_transport_sigv4(
    mcp_url: str, service_name: str, region: str
):
    """
    Create a streamable HTTP transport with AWS SigV4 authentication.

    This function creates an MCP client transport that uses AWS Signature Version 4 (SigV4)
    to authenticate requests. Essential for connecting to IAM-authenticated gateways.

    Args:
        mcp_url (str): The URL of the MCP gateway endpoint
        service_name (str): The AWS service name for SigV4 signing
        region (str): The AWS region where the gateway is deployed

    Returns:
        StreamableHTTPTransportWithSigV4: A transport instance configured for SigV4 auth
    """
    # Get AWS credentials from the current boto3 session
    # These credentials will be used to sign requests with SigV4

    session = boto3.Session()
    credentials = session.get_credentials()

    return streamablehttp_client_with_sigv4(
        url=mcp_url,
        credentials=credentials,  # Uses credentials from the Lambda execution role
        service=service_name,
        region=region,
    )


def get_full_tools_list(client):
    """
    Retrieve the complete list of tools from an MCP client, handling pagination.

    MCP servers may return tools in paginated responses. This function handles the
    pagination automatically and returns all available tools in a single list.

    Args:
        client: An MCP client instance

    Returns:
        list: A complete list of all tools available from the MCP server
    """
    more_tools = True
    tools = []
    pagination_token = None

    # Iterate through all pages of tools
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)

        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True
            pagination_token = tmp_tools.pagination_token

    return tools


GATEWAY_URL = get_required_env("GATEWAY_URL")
GATEWAY_REGION = get_required_env("GATEWAY_REGION")

# Create the MCP client with SigV4 authentication
mcp_client = MCPClient(
    lambda: create_streamable_http_transport_sigv4(
        mcp_url=GATEWAY_URL,  # Gateway URL should be set as an environment variable
        service_name="bedrock-agentcore",
        region=GATEWAY_REGION,
    )
)

# Start the MCP client connection
mcp_client.start()

# Fetch all available tools from the gateway
tools = get_full_tools_list(mcp_client)

# Create the Strands agent with the model, system prompt, and tools
agent = Agent(
    model=model,
    system_prompt=system_prompt,
    tools=tools,
)


@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Main entrypoint for the AgentCore Runtime deployed agent.

    This function is invoked when the agent receives a request through AgentCore Runtime.
    It extracts the user's prompt from the payload and returns the agent's response.

    Args:
        payload (dict): The incoming request payload containing the user's prompt
                       Expected format: {"prompt": "user's question"}

    Returns:
        str: The agent's text response after processing the prompt

    Example payload:
        {"prompt": "What is the weather on Mars?"}
    """
    # Extract the user's input from the payload
    user_input = payload.get("prompt")
    print("User input:", user_input)

    # Invoke the agent with the user's prompt
    # The agent will decide which tools to use (if any) to answer the question
    response = agent(user_input)

    # Extract and return the text content from the response
    return response.message["content"][0]["text"]


# Standard Python idiom: only run the app when this file is executed directly
if __name__ == "__main__":
    app.run()

### Step 6.2: Configure Amazon Bedrock AgentCore Runtime

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<center>

![prereq](./images/configure.png)

</center>

In [ ]:
# Initialize the AgentCore Runtime manager
# This object handles the deployment of our agent to AWS Lambda
agentcore_runtime = Runtime()

# Configure the runtime deployment settings
# This prepares all necessary AWS resources for deploying the agent
response = agentcore_runtime.configure(
    entrypoint="strands.py",  
    auto_create_ecr=True, 
    requirements_file="requirements.txt",  
    region=region,  
    agent_name=agent_name,
    execution_role=runtime_execution_role_arn,
    non_interactive=True
)

# Display the configuration response
response

### Step 6.3: Launch Amazon Bedrock AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<center>

![prereq](./images/launch.png)

</center>

In [ ]:
# Launch the agent to AWS Lambda via AgentCore Runtime
# This builds the container image, pushes it to ECR, and creates the Lambda function
# Note: This step can take several minutes as it builds and uploads the container
launch_result = agentcore_runtime.launch(env_vars={
    "GATEWAY_URL": gateway_url,
    "GATEWAY_REGION": region,
})

### Step 6.4: Invoke AgentCore runtime

Finally, we can invoke our AgentCore Runtime with a payload

<center>

![prereq](./images/invoke.png)

</center>

In [ ]:
# Test the deployed agent by invoking it with a sample prompt
# This calls the Lambda function through AgentCore Runtime
invoke_response = agentcore_runtime.invoke({"prompt": "What is the weather in northern part of the mars?"})

# Display the response from the agent
print(invoke_response["response"][0])

In [ ]:
# Test the warranty check functionality of the deployed agent
# This verifies that the Lambda tool integration is working correctly
invoke_response = agentcore_runtime.invoke({"prompt": "I have a Gaming Console Pro device , I want to check my warranty status, warranty serial number is MNO33333333."})

# Display the warranty check response
print(invoke_response["response"][0])

## Cleanup

In [ ]:
# Delete the gateway and all its associated targets
# The utils.delete_gateway function handles both target deletion and gateway deletion
# Replace the gateway ID with your actual gateway ID from the previous cell

gateways = agentcore_client.list_gateways()

# Iterate through all pages of gateways (handles pagination)
while True:
    for gateway in gateways["items"]:
        if gateway['name'] == gateway_name:
            utils.delete_gateway(agentcore_client, gateway['gatewayId'])
    
    if "nextToken" not in gateways:
        break
    else:
        gateways = agentcore_client.list_gateways(nextToken=gateways['nextToken'])

In [ ]:
# ===================================================================
# COMPREHENSIVE CLEANUP SECTION
# This cell removes all AWS resources created during the tutorial
# Run this to avoid incurring unnecessary AWS charges
# ===================================================================

# Import the cleanup function

# Step 1: Delete the API Key Credential Provider
# This removes the NASA API key from AWS Secrets Manager
try:
    logger.info(f"Deleting API Key Credential Provider: {api_key_credential_provider_name}")
    identity_client.delete_api_key_credential_provider(
        name=api_key_credential_provider_name
    )
    logger.info("Successfully deleted API Key Credential Provider")
except identity_client.exceptions.ResourceNotFoundException:
    logger.warning("API Key Credential Provider does not exist")
except Exception as e:
    logger.error(f"Error deleting API Key Credential Provider: {e}")

# Step 2: Delete the S3 bucket and all its contents
# We need to delete all objects before we can delete the bucket itself
try:
    logger.info(f"Deleting S3 bucket: {bucket_name}")
    
    objects = s3_client.list_objects_v2(Bucket=bucket_name)
    
    if 'Contents' in objects:
        for obj in objects['Contents']:
            s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
            logger.info(f"Deleted object: {obj['Key']}")
    
    s3_client.delete_bucket(Bucket=bucket_name)
    logger.info(f"Successfully deleted S3 bucket: {bucket_name}")
except s3_client.exceptions.NoSuchBucket:
    logger.warning(f"S3 bucket {bucket_name} does not exist")
except Exception as e:
    logger.error(f"Error deleting S3 bucket: {e}")

# Step 3: Delete the CloudFormation stack
# This removes the Lambda function, DynamoDB table, and IAM roles
delete_stack(
    stack_name=stack_name,
    region=region,
    cf_client=cf_client,
    wait=True
)

In [ ]:
# Step 4: Clean AgentCore Runtime and other resources

destroy_bedrock_agentcore(
    config_path=Path(".bedrock_agentcore.yaml"),
    agent_name=agent_name,
    delete_ecr_repo=True
    
)

In [ ]:
# Clean up the stored variables from Jupyter's variable store
# These were saved with %store earlier in the notebook
# Removing them prevents conflicts if you re-run the notebook 
%store -d unique_s3_name  

# Congratulations!